# Import Libraries and Mounting Drive



In [ ]:
# importing Libraries

import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
import numpy as np
import os 
from tensorflow.keras.layers import *
import glob
import random
from PIL import Image   # Python Imaging Library
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import matplotlib as mpl
import pickle


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Setting Parameters

In [ ]:
n_epochs = 10
batch_size = 8
learning_rate = 1e-4
weight_decay = 1e-4

# K estimate Module

In [ ]:
def Aod_net(X):
  
  c1 = Conv2D(3,1,1,padding="SAME",activation="relu",kernel_initializer=tf.initializers.random_normal(stddev=0.02),
                kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(X)
  
  c2 = Conv2D(3,3,1,padding="SAME",activation="relu",kernel_initializer=tf.initializers.random_normal(stddev=0.02),
                kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(c1)
  
  c1c2 = tf.concat([c1,c2],axis=-1)
  
  c3 = Conv2D(3,5,1,padding="SAME",activation="relu",kernel_initializer=tf.initializers.random_normal(stddev=0.02),
                kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(c1c2)
  
  c2c3 = tf.concat([c2,c3],axis=-1)
  
  c4 = Conv2D(3,7,1,padding="SAME",activation="relu",kernel_initializer=tf.initializers.random_normal(stddev=0.02),
                kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(c2c3)
  
  c1c2c3c4 = tf.concat([c1,c2,c3,c4],axis=-1)
  
  c5 = Conv2D(3,3,1,padding="SAME",activation="relu",kernel_initializer=tf.initializers.random_normal(stddev=0.02),
                kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(c1c2c3c4)
  
  K = c5
  
  output = ReLU(max_value=1.0)(tf.math.multiply(K,X) - K + 1.0)
  
  return output

# Data Loading & Pre-processing

In [ ]:
def setup_data_paths(orig_images_path,hazy_images_path):
  
  orig_image_paths = glob.glob(orig_images_path + "/*.jpg")
  n = len(orig_image_paths) 
  random.shuffle(orig_image_paths)
  
  # Splitting dataset into 90%-10% Split
  train_keys = orig_image_paths[:int(0.90*n)]
  val_keys = orig_image_paths[int(0.90*n):]
  
  # Creating a dictionary to map original image paths
  split_dict = {}
  for key in train_keys:
    split_dict[key] = 'train'
  for key in val_keys:
    split_dict[key] = 'val'
  
  train_data = []
  val_data = []
  

  # Splitting haze images into train and validation set
  hazy_image_paths = glob.glob(hazy_images_path + "/*.jpg")
  for path in hazy_image_paths:
    print(path)
    label = path.split('/')[-1]
    orig_path = orig_images_path + "/" + label.split('_')[0] + '_' + label.split('_')[1]
    if(split_dict[orig_path] == 'train'):
      train_data.append([path,orig_path])
    else: val_data.append([path,orig_path])
  
  return train_data, val_data

In [ ]:
def load_image(X):
  raw = tf.io.read_file(X)
  x= tf.image.decode_jpeg(raw,channels=3)
  X = tf.image.resize(x,(480,640))
  X = X / 255.0
  return X

# def showImage(x):
#   x = np.asarray(x*255,dtype=np.int32)
#   plt.figure()
#   plt.imshow(x)
#   plt.show()

In [ ]:
# Loading the images in the dataset

def create_datasets(train_data,val_data,batch_size):
  
  train_ds_hazy = tf.data.Dataset.from_tensor_slices([data[0] for data in train_data]).map(lambda x: load_image(x))
  train_ds_orig = tf.data.Dataset.from_tensor_slices([data[1] for data in train_data]).map(lambda x: load_image(x))
  train_ds = tf.data.Dataset.zip((train_ds_hazy,train_ds_orig)).shuffle(100).repeat().batch(batch_size)

  val_ds_hazy = tf.data.Dataset.from_tensor_slices([data[0] for data in val_data]).map(lambda x: load_image(x))
  val_ds_orig = tf.data.Dataset.from_tensor_slices([data[1] for data in val_data]).map(lambda x: load_image(x))
  val_ds = tf.data.Dataset.zip((val_ds_hazy,val_ds_orig)).shuffle(100).repeat().batch(batch_size)
  
  

In [ ]:
iterator = tf.data.Iterator.from_structure(tf.data.get_output_types(train_ds),tf.data.get_output_shapes(train_ds))
  
  train_init_op = iterator.make_initializer(train_ds)
  val_init_op = iterator.make_initializer(val_ds)
  
  return train_init_op, val_init_op, iterator

# Training

In [ ]:
np.random.seed(0)
ops.reset_default_graph()
train_data, val_data = setup_data_paths(orig_images_path="/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder", hazy_images_path="/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder");
train_init_op, val_init_op, iterator = create_datasets(train_data,val_data,batch_size)
next_element = iterator.get_next()

X = tf.placeholder(shape=(None,480,640,3),dtype=tf.float32)
Y = tf.placeholder(shape=(None,480,640,3),dtype=tf.float32)
dehazed_X = Aod_net(X)

loss = tf.reduce_mean(tf.square(dehazed_X-Y))  # mean squared error
optimizer = tf.train.AdamOptimizer(learning_rate)  #Learning Rate
trainable_variables = tf.trainable_variables()  #passing the list of trainable variables

/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0151.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0146.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0162.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0141.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0161.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0130.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0143.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0155.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0133.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0157.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0138.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0

Instructions for updating:
Use `tf.compat.v1.data.get_output_types(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(iterator)`.
Instructions for updating:
Use `tf.compat.v1.data.get_output_classes(iterator)`.


In [ ]:
saver = tf.train.Saver()
load_path = None

with tf.device('/gpu:0'):
  with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
      
      sess.run(train_init_op)
      batches = len(train_data) // batch_size
      epoch_loss = 0.0
      for batch in range(batches):

        batch_data = sess.run(next_element)
        batch_loss, _ = sess.run([loss,optimizer],feed_dict={X:batch_data[0],Y:batch_data[1]})
        epoch_loss += batch_loss / float(batches)
        if batch % 1000 == 0:
          print("Training loss at batch %d: %f\n"%(batch,batch_loss))
            
      train_loss = epoch_loss

      sess.run(val_init_op)
      batches= len(val_data) // batch_size
      epoch_loss = 0.0
      for batch in range(batches):
        batch_data = sess.run(next_element)
        batch_loss = sess.run(loss,feed_dict={X:batch_data[0],
                                             Y:batch_data[1]})
        epoch_loss += batch_loss / float(batches)
        if batch % 100 == 0:
          print("Validation loss at batch %d: %f\n"%(batch,batch_loss))
          for j in range(-2 + batch_size//2):
            x = batch_data[0][j].reshape((1,)+batch_data[0][j].shape)
            y = batch_data[1][j].reshape((1,)+batch_data[1][j].shape)
            dehazed_x = sess.run(dehazed_X,feed_dict={X:x,Y:y})
            print("Image Number: %d\n"%(j))
      val_loss = epoch_loss

      saver.save(sess,'./models/model_checkpoint_' + str(epoch) + '.h5')
      
      print("Epoch %d\nTraining loss: %f\nValidation loss: %f\n\n"%(epoch,train_loss,val_loss))
    

Training loss at batch 0: 0.235788

Validation loss at batch 0: 0.005171

Image Number: 0

Image Number: 1

Epoch 0
Training loss: 0.176657
Validation loss: 0.005640


Training loss at batch 0: 0.003715

Validation loss at batch 0: 0.001324

Image Number: 0

Image Number: 1

Epoch 1
Training loss: 0.002670
Validation loss: 0.001469


Training loss at batch 0: 0.001083

Validation loss at batch 0: 0.000926

Image Number: 0

Image Number: 1

Epoch 2
Training loss: 0.001219
Validation loss: 0.001166


Training loss at batch 0: 0.000749

Validation loss at batch 0: 0.000900

Image Number: 0

Image Number: 1

Epoch 3
Training loss: 0.001103
Validation loss: 0.001098


Training loss at batch 0: 0.000864

Validation loss at batch 0: 0.000731

Image Number: 0

Image Number: 1

Epoch 4
Training loss: 0.001063
Validation loss: 0.001081


Training loss at batch 0: 0.000730

Validation loss at batch 0: 0.000800

Image Number: 0

Image Number: 1



Instructions for updating:
Use standard file APIs to delete files with this prefix.


Epoch 5
Training loss: 0.001036
Validation loss: 0.001054


Training loss at batch 0: 0.000802

Validation loss at batch 0: 0.000706

Image Number: 0

Image Number: 1

Epoch 6
Training loss: 0.001014
Validation loss: 0.001024


Training loss at batch 0: 0.000771

Validation loss at batch 0: 0.000791

Image Number: 0

Image Number: 1

Epoch 7
Training loss: 0.000990
Validation loss: 0.001006


Training loss at batch 0: 0.000945

Validation loss at batch 0: 0.000808

Image Number: 0

Image Number: 1

Epoch 8
Training loss: 0.000964
Validation loss: 0.000977


Training loss at batch 0: 0.000708

Validation loss at batch 0: 0.000679

Image Number: 0

Image Number: 1

Epoch 9
Training loss: 0.000936
Validation loss: 0.000946




In [ ]:
next_element = iterator.get_next()

with tf.Session() as sess:
  sess.run(val_init_op)
  
  for i in range(10):
    batch_data = sess.run(next_element)
    for j in range(4):
      x = batch_data[0][j].reshape((1,)+batch_data[0][j].shape)

# Evaluation

In [ ]:
tf.reset_default_graph()
train_data, val_data = setup_data_paths(orig_images_path="/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder",
                                        hazy_images_path = "/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder");
train_init_op, val_init_op, iterator = create_datasets(train_data,val_data,batch_size)
next_element = iterator.get_next()

X = tf.placeholder(shape=(None,480,640,3),dtype=tf.float32)
Y = tf.placeholder(shape=(None,480,640,3),dtype=tf.float32)
dehazed_X = Aod_net(X)

loss = tf.reduce_mean(tf.square(dehazed_X-Y))
optimizer = tf.train.AdamOptimizer(learning_rate)
trainable_variables = tf.trainable_variables()
gradients_and_vars = optimizer.compute_gradients(loss,trainable_variables)
clipped_gradients = [(tf.clip_by_norm(gradient,0.1),var) for gradient,var in gradients_and_vars]
optimizer = optimizer.apply_gradients(gradients_and_vars)

/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0151.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0146.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0162.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0141.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0161.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0130.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0143.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0155.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0133.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0157.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0138.jpg
/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/New folder/haze_0

# Saving Final Results

In [ ]:
import pickle
pickle_out = open("img_model.pkl", mode = "wb") 
pickle.dump('/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/model_checkpoint_9.h5.data-00000-of-00001', pickle_out) 
pickle_out.close()

In [ ]:
saver = tf.train.Saver()


test_input_folder = "/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/Result/tests"
test_output_folder = "/content/gdrive/MyDrive/Colab Notebooks/DL Lab/Fog Removal/Result/dehazed_test_images"
if not os.path.exists(test_output_folder):
  os.mkdir(test_output_folder)
  
file_types = ['jpeg','jpg']

with tf.Session() as sess:
  saver.restore(sess,'/content/models/model_checkpoint_9.h5')
  test_image_paths = []
  for file_type in file_types:
    test_image_paths.extend(glob.glob(test_input_folder+"/*."+file_type))
  
  
  for path in test_image_paths:
    image_label = path.split(test_input_folder)[-1][1:]
    image = Image.open(path)
    image = image.resize((640, 480))
    image = np.asarray(image) / 255.0
    image = image.reshape((1,) + image.shape)
    dehazed_image = sess.run(dehazed_X,feed_dict={X:image,Y:image})
    
    
    dehazed_image = np.asarray(dehazed_image[0] * 255,dtype=np.uint8)
    mpl.image.imsave(test_output_folder + "/" + 'dehazed_' + image_label, dehazed_image)